## Поиск похожих товаров и пользователей. Гибридные рекомендательные системы

In [ ]:
# !pip install lightfm

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import itertools

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
  
# from metrics import precision_at_k, recall_at_k
from utils import prefilter_items

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [2]:
data = pd.read_csv('data/retail_train.csv')
item_features = pd.read_csv('data/product.csv')
user_features = pd.read_csv('data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [4]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


### Filter items

In [5]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


### Prepare data set

In [6]:
user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(2)

item_id,25671,26081,26093,26190,26355,26426,26540,26601,26636,26691,...,17328742,17329473,17329749,17330255,17330511,17381856,17382205,17383227,17827644,17829232
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())] #только те, кто есть в трейне!!
test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) # необходимый тип матрицы для implicit

In [8]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

### Prepare user and item features

In [9]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)
user_feat.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
user_id,,,,,,,
1,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)
item_feat.head(2)

,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
item_id,,,,,,
25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [11]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [12]:
user_feat_lightfm.head(2)

,age_desc_19-24,age_desc_25-34,age_desc_35-44,age_desc_45-54,age_desc_55-64,age_desc_65+,marital_status_code_A,marital_status_code_B,marital_status_code_U,income_desc_100-124K,...,hh_comp_desc_Unknown,household_size_desc_1,household_size_desc_2,household_size_desc_3,household_size_desc_4,household_size_desc_5+,kid_category_desc_1,kid_category_desc_2,kid_category_desc_3+,kid_category_desc_None/Unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Init model

In [13]:
model = LightFM(no_components=30,
#                 loss='bpr',
                loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

In [14]:
%%time
model.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          
          epochs=15, 
          num_threads=4,
          verbose=False)

Wall time: 5min 3s


In [15]:
user_emb = model.get_user_representations(features=csr_matrix(user_feat_lightfm.values).tocsr())

In [16]:
user_emb[0].shape

(2499,)

In [17]:
user_emb[1]

array([[-3.7401859e-11,  4.4659870e-10, -6.6604033e-10, ...,
         3.7958703e-10, -3.9236892e-11, -1.0067969e-10],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       ...,
       [-6.3674754e-10, -5.9009864e-10, -2.6816793e-10, ...,
         4.3804205e-10, -2.7498598e-10, -3.0505376e-11],
       [-9.6898378e-10, -3.2080294e-10, -9.5991504e-10, ...,
         7.7631235e-12,  3.0306313e-11,  9.7277963e-11],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)

In [18]:
user_emb[1].shape

(2499, 30)

In [19]:
item_emb = model.get_item_representations(features=csr_matrix(item_feat_lightfm.values).tocsr())

In [20]:
item_emb[0].shape

(86865,)

In [21]:
item_emb[1].shape

(86865, 30)

In [22]:
item_emb[1]

array([[-6.67844660e-08,  1.13940946e-08, -2.33089963e-07, ...,
         8.19502617e-08, -1.15997665e-07, -7.61727392e-08],
       [ 7.73582940e-08,  2.18644054e-08,  1.43830064e-07, ...,
        -1.54109845e-08, -5.50294343e-09,  1.02144554e-07],
       [-8.85180711e-08, -2.39238478e-08,  9.43829548e-09, ...,
        -5.91453428e-08,  4.71297525e-08,  1.75157897e-08],
       ...,
       [-7.03749592e-08,  1.25145903e-07,  3.51136805e-07, ...,
        -2.23867147e-07,  5.04839875e-07,  3.39319286e-07],
       [ 1.31595399e-07,  2.26507808e-08,  1.62007154e-07, ...,
        -3.20939697e-09, -3.95994633e-08,  8.53275708e-08],
       [ 4.39171401e-07, -1.26780961e-07,  4.74430720e-07, ...,
         1.83793887e-07,  2.84737098e-07,  5.63166012e-08]], dtype=float32)

In [23]:
train_precision = precision_at_k(model, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision

0.051700678

In [24]:
test_item_ids = np.array([1, 2, 3, 200, 1200])

predictions = model.predict(user_ids=0, item_ids=test_item_ids,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=4)

In [25]:
predictions

array([-70.98138 , -70.50838 , -70.52468 , -70.79794 , -70.499954],
      dtype=float32)

In [26]:
csr_matrix(test_user_item_matrix).tocsr()

<2041x22143 sparse matrix of type '<class 'numpy.float64'>'
	with 102088 stored elements in Compressed Sparse Row format>

In [27]:
test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

test_precision

0.0022537974

Результат стал хуже

### Model bpr

In [28]:
model_bpr = LightFM(no_components=30,
                loss='bpr',
#                 loss='warp',
                learning_rate=0.05, 
                item_alpha=0.1,
                user_alpha=0.1, 
                random_state=42)

In [29]:
%%time
model_bpr.fit((sparse_user_item > 0) * 1,  # user-item matrix из 0 и 1
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=15, 
          num_threads=4,
          verbose=False)

Wall time: 4min 16s


In [30]:
train_precision_bpr = precision_at_k(model_bpr, sparse_user_item, 
                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                 k=5).mean()

train_precision_bpr

0.0

С __bpr__ метрика = 0, значит, не подходит!

### Params search

In [31]:
l2 = [0.01, 0.05, 0.1]
learning_rate = [0.01, 0.05, 0.1]
max_sampled = [7, 10, 12]

In [32]:
%%time
metrics = {}
for l, lr, ms in itertools.product(l2, learning_rate, max_sampled):
    print(l, lr, ms)
    
    model = LightFM(no_components=30, loss='warp', item_alpha=l, user_alpha=l, random_state=42, learning_rate = lr, max_sampled = ms)
    
    model.fit((sparse_user_item > 0) * 1, sample_weight=coo_matrix(user_item_matrix), user_features=csr_matrix(user_feat_lightfm.values).tocsr(), item_features=csr_matrix(item_feat_lightfm.values).tocsr(), epochs=10, num_threads=4, verbose=10) 
    
    test_precision = precision_at_k(model, csr_matrix(test_user_item_matrix).tocsr(), user_features=csr_matrix(user_feat_lightfm.values).tocsr(), item_features=csr_matrix(item_feat_lightfm.values).tocsr(), k=5).mean()
    metrics[str(l) + str(lr) + str(ms)] = test_precision

0.01 0.01 7


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [03:02<00:00, 18.29s/it]


0.01 0.01 10


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [03:08<00:00, 18.82s/it]


0.01 0.01 12


...

0.1 0.1 10


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [02:56<00:00, 17.68s/it]


0.1 0.1 12


Epoch: 100%|███████████████████████████████████████████████████████████████████████████| 10/10 [02:59<00:00, 17.91s/it]


Wall time: 1h 41min 5s


In [34]:
pd.DataFrame.from_dict(metrics, orient='index', columns = ['prc']).sort_values(by = 'prc', ascending = False).head(10)

,prc
0.050.0510,0.003234
0.050.057,0.003234
0.10.0512,0.003234
0.10.0510,0.003234
0.10.057,0.003234
0.050.0512,0.003234
0.050.110,0.002940
0.010.110,0.002744
0.010.017,0.002646
0.010.112,0.002548


Получили прирост presicion на l2